# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
! python --version

Python 3.6.3


In [2]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType


In [3]:
# create a Spark session
spark = SparkSession.builder \
    .master("local") \
    .appName("Sparkify") \
    .getOrCreate()

In [4]:
spark.sparkContext.getConf().getAll()

[('spark.master', 'local'),
 ('spark.app.id', 'local-1571844451087'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.name', 'Sparkify'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.port', '43225'),
 ('spark.driver.host', '4a13605e1578'),
 ('spark.ui.showConsoleProgress', 'true')]

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [5]:
sparkify_data_file = "./mini_sparkify_event_data.json"
event_data = spark.read.json(sparkify_data_file)
event_data.persist()

DataFrame[artist: string, auth: string, firstName: string, gender: string, itemInSession: bigint, lastName: string, length: double, level: string, location: string, method: string, page: string, registration: bigint, sessionId: bigint, song: string, status: bigint, ts: bigint, userAgent: string, userId: string]

In [6]:
event_data.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')

In [7]:
event_data.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [8]:
event_data.select("userId").dropDuplicates().sort("userId").show()

+------+
|userId|
+------+
|      |
|    10|
|   100|
|100001|
|100002|
|100003|
|100004|
|100005|
|100006|
|100007|
|100008|
|100009|
|100010|
|100011|
|100012|
|100013|
|100014|
|100015|
|100016|
|100017|
+------+
only showing top 20 rows



In [9]:
event_data.select("sessionID").dropDuplicates().sort("sessionID").show()

+---------+
|sessionID|
+---------+
|        1|
|        2|
|        3|
|        4|
|        5|
|        6|
|        7|
|        8|
|        9|
|       10|
|       11|
|       12|
|       13|
|       15|
|       16|
|       17|
|       18|
|       19|
|       20|
|       21|
+---------+
only showing top 20 rows



In [10]:
event_data.filter(event_data["userId"] == "").count()

8346

In [11]:
event_data.filter(event_data["sessionId"] == "").count()

0

In [12]:
user_log_valid = event_data.dropna(how = "any", subset = ["userId", "sessionId"])

In [13]:
user_log_valid.count()

286500

In [14]:
user_log_valid = user_log_valid.filter(user_log_valid["userId"] != "")

In [15]:
user_log_valid.count()

278154

In [16]:
286500-278154

8346

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [17]:
# define churn looking at 'cancellation confirmation' event


In [18]:
user_log_valid.select("page").dropDuplicates().collect()

[Row(page='Cancel'),
 Row(page='Submit Downgrade'),
 Row(page='Thumbs Down'),
 Row(page='Home'),
 Row(page='Downgrade'),
 Row(page='Roll Advert'),
 Row(page='Logout'),
 Row(page='Save Settings'),
 Row(page='Cancellation Confirmation'),
 Row(page='About'),
 Row(page='Settings'),
 Row(page='Add to Playlist'),
 Row(page='Add Friend'),
 Row(page='NextSong'),
 Row(page='Thumbs Up'),
 Row(page='Help'),
 Row(page='Upgrade'),
 Row(page='Error'),
 Row(page='Submit Upgrade')]

In [19]:
user_log_valid \
.select(["userId", "sessionId", "firstname", "page", "song"]) \
.where(user_log_valid.page == "Cancellation Confirmation") \
.collect()

[Row(userId='18', sessionId=514, firstname='Adriel', page='Cancellation Confirmation', song=None),
 Row(userId='32', sessionId=540, firstname='Diego', page='Cancellation Confirmation', song=None),
 Row(userId='125', sessionId=174, firstname='Mason', page='Cancellation Confirmation', song=None),
 Row(userId='105', sessionId=508, firstname='Alexander', page='Cancellation Confirmation', song=None),
 Row(userId='17', sessionId=797, firstname='Kayla', page='Cancellation Confirmation', song=None),
 Row(userId='143', sessionId=843, firstname='Molly', page='Cancellation Confirmation', song=None),
 Row(userId='101', sessionId=842, firstname='Alex', page='Cancellation Confirmation', song=None),
 Row(userId='129', sessionId=802, firstname='Davis', page='Cancellation Confirmation', song=None),
 Row(userId='121', sessionId=881, firstname='Nikolas', page='Cancellation Confirmation', song=None),
 Row(userId='51', sessionId=934, firstname='Ethan', page='Cancellation Confirmation', song=None),
 Row(use

In [37]:
user_log_valid \
.select(["userId", "sessionId", "firstname", "page", "song"]) \
.where(user_log_valid.page == "Submit Downgrade") \
.collect()

[Row(userId='131', sessionId=249, firstname='Kael', page='Submit Downgrade', song=None),
 Row(userId='38', sessionId=313, firstname='Calvin', page='Submit Downgrade', song=None),
 Row(userId='141', sessionId=479, firstname='Kaylenn', page='Submit Downgrade', song=None),
 Row(userId='30', sessionId=532, firstname='Colin', page='Submit Downgrade', song=None),
 Row(userId='20', sessionId=378, firstname='Destiny', page='Submit Downgrade', song=None),
 Row(userId='12', sessionId=632, firstname='Sophia', page='Submit Downgrade', song=None),
 Row(userId='11', sessionId=487, firstname='Zoe', page='Submit Downgrade', song=None),
 Row(userId='61', sessionId=529, firstname='Parker', page='Submit Downgrade', song=None),
 Row(userId='81', sessionId=725, firstname='Oliver', page='Submit Downgrade', song=None),
 Row(userId='85', sessionId=734, firstname='Brayden', page='Submit Downgrade', song=None),
 Row(userId='140', sessionId=753, firstname='Lucero', page='Submit Downgrade', song=None),
 Row(userI

In [26]:
flag_downgrade_event = udf(lambda x: 1 if x == "Submit Downgrade" else 0, IntegerType())
flag_cancellation_event = udf(lambda x: 1 if x == "Cancellation Confirmation" else 0, IntegerType())

In [27]:
user_log_valid = user_log_valid.withColumn("downgrade", flag_downgrade_event("page"))
user_log_valid = user_log_valid.withColumn("churn", flag_cancellation_event("page"))

In [28]:
user_log_valid.head(1)

[Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30', churn=0, downgrade=0)]

In [33]:
user_log_valid \
.select(["userId", "ts", "page", "level", "churn", "downgrade"]) \
.filter(user_log_valid.userId == "18").filter(user_log_valid.churn == "1") \
.sort("ts") \
.collect()

[Row(userId='18', ts=1538943990000, page='Cancellation Confirmation', level='paid', churn=1, downgrade=0)]

In [41]:
user_log_valid \
.select(["userId", "ts", "page", "level", "churn", "downgrade"]) \
.filter(user_log_valid.downgrade == "1") \
.sort("ts") \
.show(user_log_valid.count(),False)

+------+-------------+----------------+-----+-----+---------+
|userId|ts           |page            |level|churn|downgrade|
+------+-------------+----------------+-----+-----+---------+
|131   |1538393619000|Submit Downgrade|paid |0    |1        |
|38    |1538516445000|Submit Downgrade|paid |0    |1        |
|200003|1538744281000|Submit Downgrade|paid |0    |1        |
|200023|1538753821000|Submit Downgrade|paid |0    |1        |
|141   |1538835479000|Submit Downgrade|paid |0    |1        |
|30    |1538995454000|Submit Downgrade|paid |0    |1        |
|20    |1538997356000|Submit Downgrade|paid |0    |1        |
|300015|1539027805000|Submit Downgrade|paid |0    |1        |
|100018|1539106390000|Submit Downgrade|paid |0    |1        |
|12    |1539144047000|Submit Downgrade|paid |0    |1        |
|11    |1539239742000|Submit Downgrade|paid |0    |1        |
|61    |1539244188000|Submit Downgrade|paid |0    |1        |
|81    |1539264337000|Submit Downgrade|paid |0    |1        |
|200025|

In [ ]:
# find the users who have churn and downgraded


In [42]:
downgrade_logs = user_log_valid.filter(user_log_valid.downgrade == "1")
downgrade_logs.select(['userID','page', 'firstName', 'lastName','ts', 'auth']).show(5, False)

+------+----------------+---------+--------+-------------+---------+
|userID|page            |firstName|lastName|ts           |auth     |
+------+----------------+---------+--------+-------------+---------+
|131   |Submit Downgrade|Kael     |Baker   |1538393619000|Logged In|
|38    |Submit Downgrade|Calvin   |Marshall|1538516445000|Logged In|
|141   |Submit Downgrade|Kaylenn  |Jenkins |1538835479000|Logged In|
|30    |Submit Downgrade|Colin    |Freeman |1538995454000|Logged In|
|20    |Submit Downgrade|Destiny  |Herring |1538997356000|Logged In|
+------+----------------+---------+--------+-------------+---------+
only showing top 5 rows



In [46]:
churn_logs = user_log_valid.filter(user_log_valid.churn == "1")
churn_logs.select(['userID','page', 'firstName', 'lastName','ts', 'auth']).show(5, False)

+------+-------------------------+---------+--------+-------------+---------+
|userID|page                     |firstName|lastName|ts           |auth     |
+------+-------------------------+---------+--------+-------------+---------+
|18    |Cancellation Confirmation|Adriel   |Mendoza |1538943990000|Cancelled|
|32    |Cancellation Confirmation|Diego    |Mckee   |1539033046000|Cancelled|
|125   |Cancellation Confirmation|Mason    |Hart    |1539318918000|Cancelled|
|105   |Cancellation Confirmation|Alexander|Garcia  |1539375441000|Cancelled|
|17    |Cancellation Confirmation|Kayla    |Johnson |1539465584000|Cancelled|
+------+-------------------------+---------+--------+-------------+---------+
only showing top 5 rows



In [62]:
user_ids_who_churn = [user_ids['userID'] for user_ids in churn_logs.select('userID').distinct().collect()]

In [80]:
user_ids_who_downgraded = [user_ids['userID'] for user_ids in downgrade_logs.select('userID').distinct().collect()]

In [63]:
len(user_ids_who_churn)

52

In [68]:
len(user_ids_who_downgraded)

63

In [82]:
user_ids_who_did_both = [x for x in user_ids_who_churn if x in user_ids_who_downgraded]

In [83]:
user_ids_who_did_both

['12', '103', '54', '100025', '100009', '100012', '100015', '200011', '200020']

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.